## Simple Iterators

### Basic usage

A very basic iterator can be defined using a yield inside what looks like a function. When a yield is present inside a `def` block, the result is no longer a function but what's known as a `generator`.

By calling the generator, it returns a generator object. This generator object is inherently an iterator.

In [4]:
def get_odds():
    for i in range(2):
        yield i * 2 + 1


get_odds()  # instead of returning a value, a generator object is returned


<generator object get_odds at 0x7ff0604aef80>

> **What does yield mean?**

> You can interpret yield in many ways. In this case the `yield` is similar to a `return` where a value is passed back to the caller.
> The key difference is instead of simply concluding in the case of return, with yield the caller can resume the generator.



To fetch values that are yielded by a `generator` we can use the python builtin `next` function.

In [5]:
odds = get_odds()

print(next(odds))
print(next(odds))

1
3


Fetching when the iterator is empty will result in a `StopIteration` error.

In [6]:
print(next(odds))

StopIteration: 

Easiest way to get values out of a generator object is by iterating it. This can be done in any number of ways: for-loop, turning it into a data structure etc.

In [7]:
for i in get_odds():
    print(i)

print(list(get_odds()))

1
3
[1, 3]


### The Object Model
The `get_odds` generator is actually equivalent to the following class:

In [3]:
from typing import Iterator


class Odds:
    def __int__(self):
        self.value = 1

    def __iter__(self):  # Required by Iterator
        """Required by the `Iterable` protocol.

        Anything that can be iterated requires this method.
        """
        return self

    def __next__(self):  # Required by Iterator
        """Makes the `Iterable` a `Iterator`.
        """
        value = self.value
        self.value += 2
        return value


Here both `get_odds` and `Odds` implement the [`Iterator` protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types).

In [9]:
from typing import Iterator

isinstance(Odds(), Iterator)
isinstance(get_odds(), Iterator)


True

#### Iterable vs Iterator

Semantically Iterables are any object that can be iterated, but iterators are the actual objects handles the state and generation of the values iterated.

A `list` or `dict` can be iterated using a for-loop so they are iterable but they themselves are not iterators.

In [12]:
from typing import Iterable, Iterator

a = [1, 2, 3, 4]
print(isinstance(a, Iterator))
print(isinstance(a, Iterable))

False
True


An Iterable contains an `__iter__` method that returns the actual iterator, this can be accessed more conveniently with python's builtin `iter` function.

In [13]:
iterator = iter(a)
print(f"{iterator = }")
print(f"{isinstance(iterator, Iterator)}")

iterator = <list_iterator object at 0x7ff0698d8760>
True


An Iterator contains a `__next__` method that returns the next value in the iterator, accessed using `next` function as shown previously. It also has an `__iter__` method, that returns itself.

A rule to remember is that an Iterator is always Iterable but not necessarily the other way round.

In [14]:
print(f"{isinstance(iterator, Iterable) = }")

True


### Unending iterators

There's no reason an iterator actually has to end. An example of this is `itertools.count(0)` which counts upwards forever. We could also write our own iterator like this:

In [ ]:
def fib():
    n = 0
    m = 1
    while True:
        n, m = m, n + m
        yield n

In [ ]:
import time

for n in fib():
    print(f"Fib: {n}")
    time.sleep(0.2)


Fib: 1
Fib: 1
Fib: 2
Fib: 3
Fib: 5
Fib: 8
Fib: 13
Fib: 21
Fib: 34
Fib: 55
Fib: 89
Fib: 144
Fib: 233
Fib: 377
Fib: 610
Fib: 987
Fib: 1597
Fib: 2584
Fib: 4181
Fib: 6765
Fib: 10946
Fib: 17711
Fib: 28657
Fib: 46368
Fib: 75025
Fib: 121393
Fib: 196418
Fib: 317811
Fib: 514229
Fib: 832040
Fib: 1346269
Fib: 2178309
Fib: 3524578


KeyboardInterrupt: ignored

# Context managers

`yield` is useful not only for generating values for an iterator, it can be viewed as a `breakpoint` for the generator. This is very useful for spliting a generator into two parts of logic. This maps very well to the concept of a context manager.

The logic before the `yield` can be mapped to `__enter__` and the logic after is the `__exit__`.

In [15]:
def calc(a: int, b: int):
    print("About to calculate a + b")
    result = a + b
    yield result
    print("Performing some clean up")


In [ ]:
calc_iter = calc(1, 1)

In [ ]:
result = next(calc_iter)
print(f"The result is {result}")

About to calculate a + b
The result is 2


In [ ]:
# Raises an error
next(calc_iter)

Performing some clean up


StopIteration: ignored

We can now turn this into a context manager. 

In [17]:
class CalcContext:
    def __init__(self, a: int, b: int) -> None:
        self.iter = calc(a, b)

    def __enter__(self):
        return next(self.iter)

    def __exit__(self):
        try:
            next(self.iter)
        except StopAsyncIteration:
            return

This is exactly what the `contextmanager` decorator does. 

In [18]:
from contextlib import contextmanager


@contextmanager
def calc_v2(a: int, b: int) -> Iterator[int]:
    print("About to calculate a + b")
    result = a + b
    yield result
    print("Performing some clean up")

## Sending

What if we want to provide input to our generator object? Generators have a `send` method that allows the caller to pass values into the generator. 

In [19]:
# Same as before but takes input from the yield

def calc_gen():
    print("About to calculate a + b")
    a, b = yield
    print(f"a = {a}, b = {b}")
    result = a + b
    yield result
    print("Performing some clean up")


c = calc_gen()

As before we can use ```next(c)``` to advance to the first yield point:

In [20]:
next(c)

About to calculate a + b


`next(c)` is equivalent to `c.send(None)`.

What happens when we send an actual value? 

In [21]:
result = c.send((1, 1))  # type: ignore

a = 1, b = 1


In [22]:
print(f"Result is {result}")

Result is 2


## Event Loop

In order to interact with the event loop we require something to track the progress and store the result.

#### The Future Object
The future object is a placeholder for the real value, when the value becomes available it is added to the future.

In [23]:
class Future:
    def __init__(self):
        self.result = None

    def set_result(self, value):
        self.result = value


Here's an example `Connection` client that uses the `Future`

In [25]:
class Connection:
    def __init__(self):
        self._futures = []

    def fetch(self):
        f = Future()
        self._futures.append(f)
        # Makes non-blocking call
        return f

    def on_receive(self, value):
        self._futures.pop(0).set_result(value)

#### Coroutine
Create a generator that `yield`s the `Future`s.

In [26]:
conn = Connection()


def do_work():
    a, b = yield (conn.fetch(), conn.fetch())
    return a + b


In [27]:
generator = do_work()

Get a hold fo the futures the coroutine depends on by calling next on the generator. 

In [28]:
f1, f2 = next(generator)
assert f1.result is None
assert f2.result is None

Once the connection comes back witha result, the event loop will set a result.

In [29]:
conn.on_receive(1)
conn.on_receive(2)

assert f1.result == 1
assert f2.result == 2

Send the results back to the generator following receiving the result.

In [30]:
try:
    generator.send((f1.result, f2.result))  # type: ignore
except StopIteration as e:
    print(f"Result is {e.value}")

Result is 3
